This is version 9.3, featuring only DCGAN

### Libraries

In [8]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Keras modules
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, BatchNormalization, Activation, ZeroPadding2D, LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, save_img

### Data preparation

In [9]:
pneumonia_dataset_dir = 'chest_xray/train/PNEUMONIA'
pneumonia_images = os.listdir(pneumonia_dataset_dir)

def get_data(data_path):
    x = []
    
    for filename in data_path:
        img = img_to_array(load_img(pneumonia_dataset_dir + "/" + filename, target_size=(56,56,3)))
        x.append(img)
        
    x = np.array(x).astype('float32')
    x = (x - 127.5) / 127.5
    
    return x

pneumonia_dataset = get_data(pneumonia_images)

### Define DC GAN

In [14]:
class DCGAN():
    def __init__(self):
        self.img_rows = 56
        self.img_cols = 56
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        self.history = []

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
                                   optimizer=optimizer,
                                   metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense((128*7*7), activation='relu', input_dim=self.latent_dim))
        model.add(Reshape((7,7,128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation('relu'))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation('relu'))
        model.add(Conv2D(self.channels, kernel_size=3, padding='same'))
        model.add(Activation('tanh'))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()
        
        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(64, kernel_size=3, strides=2, padding='same'))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(128, kernel_size=3, strides=2, padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(256, kernel_size=3, strides=2, padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        #X_train = normal_dataset
        X_train = pneumonia_dataset

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, D acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)
            
            self.history.append({"D": d_loss[0], "G": g_loss})
    
    def _predict_noise(self, size):
        noise = np.random.normal(0, 1, (size, self.latent_dim))
        return self.generator.predict(noise)
        
    def gen(self, size):
        generated = self._predict_noise(size)
        generated = 0.5 * generated + 0.5
        return generated

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("output/%d_pneumonia.png" % epoch)
        plt.close()
    
    def plot_loss(self):
        hist = pd.DataFrame(self.history)
        plt.figure(figsize=(20,5))
        for colnm in hist.columns:
            plt.plot(hist[colnm],label=colnm)
        plt.legend()
        plt.ylabel("loss")
        plt.xlabel("epochs")
        plt.show()

### Train GAN

In [ ]:
if __name__ == '__main__':
    dcgan = DCGAN()
    dcgan.train(epochs=3000, batch_size=32, sample_interval=500)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 14, 14, 32)        896       
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_10 (Batc (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 8, 8, 64)         

63 [D loss: 0.147075, D acc.: 95.31%] [G loss: 3.245283]
64 [D loss: 0.091895, D acc.: 98.44%] [G loss: 2.322093]
65 [D loss: 0.130595, D acc.: 96.88%] [G loss: 2.681814]
66 [D loss: 0.127128, D acc.: 96.88%] [G loss: 2.962873]
67 [D loss: 0.139677, D acc.: 96.88%] [G loss: 2.264380]
68 [D loss: 0.071049, D acc.: 100.00%] [G loss: 3.237006]
69 [D loss: 0.322705, D acc.: 82.81%] [G loss: 2.009206]
70 [D loss: 0.063029, D acc.: 100.00%] [G loss: 3.239723]
71 [D loss: 0.203324, D acc.: 92.19%] [G loss: 2.317349]
72 [D loss: 0.070942, D acc.: 100.00%] [G loss: 2.147605]
73 [D loss: 0.076090, D acc.: 98.44%] [G loss: 1.826568]
74 [D loss: 0.048102, D acc.: 100.00%] [G loss: 3.406927]
75 [D loss: 0.300353, D acc.: 89.06%] [G loss: 2.195366]
76 [D loss: 0.119773, D acc.: 96.88%] [G loss: 2.313711]
77 [D loss: 0.086974, D acc.: 96.88%] [G loss: 1.918263]
78 [D loss: 0.065219, D acc.: 100.00%] [G loss: 2.994929]
79 [D loss: 0.053107, D acc.: 100.00%] [G loss: 2.679245]
80 [D loss: 0.036096, D a

205 [D loss: 0.002279, D acc.: 100.00%] [G loss: 1.150086]
206 [D loss: 0.003998, D acc.: 100.00%] [G loss: 1.199035]
207 [D loss: 0.001146, D acc.: 100.00%] [G loss: 1.180697]
208 [D loss: 0.001998, D acc.: 100.00%] [G loss: 1.026770]
209 [D loss: 0.002702, D acc.: 100.00%] [G loss: 0.906831]
210 [D loss: 0.002300, D acc.: 100.00%] [G loss: 0.956424]
211 [D loss: 0.002332, D acc.: 100.00%] [G loss: 0.926009]
212 [D loss: 0.002959, D acc.: 100.00%] [G loss: 1.249742]
213 [D loss: 0.002965, D acc.: 100.00%] [G loss: 1.148209]
214 [D loss: 0.002277, D acc.: 100.00%] [G loss: 1.436892]
215 [D loss: 0.013412, D acc.: 100.00%] [G loss: 1.294437]
216 [D loss: 0.003637, D acc.: 100.00%] [G loss: 1.551599]
217 [D loss: 0.003381, D acc.: 100.00%] [G loss: 0.744566]
218 [D loss: 0.002687, D acc.: 100.00%] [G loss: 0.788294]
219 [D loss: 0.005321, D acc.: 100.00%] [G loss: 1.187356]
220 [D loss: 0.003274, D acc.: 100.00%] [G loss: 0.862219]
221 [D loss: 0.002778, D acc.: 100.00%] [G loss: 0.93707

345 [D loss: 0.000310, D acc.: 100.00%] [G loss: 0.480684]
346 [D loss: 0.000761, D acc.: 100.00%] [G loss: 1.000230]
347 [D loss: 0.000563, D acc.: 100.00%] [G loss: 0.611677]
348 [D loss: 0.000945, D acc.: 100.00%] [G loss: 0.696793]
349 [D loss: 0.000527, D acc.: 100.00%] [G loss: 0.755435]
350 [D loss: 0.000222, D acc.: 100.00%] [G loss: 0.822888]
351 [D loss: 0.000336, D acc.: 100.00%] [G loss: 0.817652]
352 [D loss: 0.000973, D acc.: 100.00%] [G loss: 0.804296]
353 [D loss: 0.003019, D acc.: 100.00%] [G loss: 0.557117]
354 [D loss: 0.000439, D acc.: 100.00%] [G loss: 0.657127]
355 [D loss: 0.000919, D acc.: 100.00%] [G loss: 0.682068]
356 [D loss: 0.000972, D acc.: 100.00%] [G loss: 0.707266]
357 [D loss: 0.000443, D acc.: 100.00%] [G loss: 0.794855]
358 [D loss: 0.000319, D acc.: 100.00%] [G loss: 0.573162]
359 [D loss: 0.000359, D acc.: 100.00%] [G loss: 0.826278]
360 [D loss: 0.000486, D acc.: 100.00%] [G loss: 0.828991]
361 [D loss: 0.000243, D acc.: 100.00%] [G loss: 0.50582

485 [D loss: 0.000184, D acc.: 100.00%] [G loss: 0.453803]
486 [D loss: 0.000218, D acc.: 100.00%] [G loss: 0.568691]
487 [D loss: 0.000318, D acc.: 100.00%] [G loss: 0.465893]
488 [D loss: 0.000357, D acc.: 100.00%] [G loss: 0.415803]
489 [D loss: 0.000145, D acc.: 100.00%] [G loss: 0.392200]
490 [D loss: 0.000191, D acc.: 100.00%] [G loss: 0.474132]
491 [D loss: 0.000167, D acc.: 100.00%] [G loss: 0.404289]
492 [D loss: 0.000343, D acc.: 100.00%] [G loss: 0.294563]
493 [D loss: 0.000124, D acc.: 100.00%] [G loss: 0.581380]
494 [D loss: 0.000206, D acc.: 100.00%] [G loss: 0.376355]
495 [D loss: 0.000414, D acc.: 100.00%] [G loss: 0.511281]
496 [D loss: 0.000341, D acc.: 100.00%] [G loss: 0.503982]
497 [D loss: 0.000234, D acc.: 100.00%] [G loss: 0.397968]
498 [D loss: 0.000211, D acc.: 100.00%] [G loss: 0.440587]
499 [D loss: 0.000146, D acc.: 100.00%] [G loss: 0.380180]
500 [D loss: 0.000252, D acc.: 100.00%] [G loss: 0.462833]
501 [D loss: 0.000240, D acc.: 100.00%] [G loss: 0.41930

...

6563 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.443308]
6564 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.688044]
6565 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.556607]
6566 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.607880]
6567 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.725373]
6568 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.511843]
6569 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.559889]
6570 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.586448]
6571 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.828141]
6572 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.821363]
6573 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.706011]
6574 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.526371]
6575 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.720662]
6576 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.482589]
6577 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.649735]
6578 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.605282]
6579 [D loss: 0.000000, D acc.: 100.00%]

6701 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.726126]
6702 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.631965]
6703 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.436864]
6704 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.758544]
6705 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.697469]
6706 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.706321]
6707 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.578665]
6708 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.461785]
6709 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.573315]
6710 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.627589]
6711 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.537274]
6712 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.667537]
6713 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.729140]
6714 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.558773]
6715 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.707498]
6716 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.864167]
6717 [D loss: 0.000000, D acc.: 100.00%]

6839 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.538087]
6840 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.555371]
6841 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.612795]
6842 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.560804]
6843 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.795023]
6844 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.793974]
6845 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.689218]
6846 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.557261]
6847 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.888713]
6848 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.954200]
6849 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.608259]
6850 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.780465]
6851 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.489235]
6852 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.540577]
6853 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.430425]
6854 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.499312]
6855 [D loss: 0.000001, D acc.: 100.00%]

6977 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.677871]
6978 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.561455]
6979 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.889664]
6980 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.395231]
6981 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.745445]
6982 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.535053]
6983 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.671733]
6984 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.709827]
6985 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.802906]
6986 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.647815]
6987 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.663449]
6988 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.513807]
6989 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.786726]
6990 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.583342]
6991 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.484938]
6992 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.451546]
6993 [D loss: 0.000000, D acc.: 100.00%]

7115 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.490384]
7116 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.699938]
7117 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.702681]
7118 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.798098]
7119 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.965210]
7120 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.673530]
7121 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.667599]
7122 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.817363]
7123 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.834613]
7124 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.580307]
7125 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.844051]
7126 [D loss: 0.000000, D acc.: 100.00%] [G loss: 0.596531]


### DC GAN loss

In [ ]:
dcgan.plot_loss()

### Generate images

In [ ]:
#gan_images = dcgan.gen(4273)
gan_images = dcgan.gen(100)

for i in range(len(gan_images)):
    #save_img("output/gan_normal/{}.png".format(i), gan_images[i])
    save_img("output/gan_pneumonia/{}.png".format(i+3000), gan_images[i])